In [40]:
import pandas as pd
import numpy as np

df_origin = pd.read_excel('all_data_with_odds.xlsx')
df = df_origin.copy()
df.head()


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,PlayerA,PlayerB,RankA,RankB,RankDiff,WinnerBinary,MaxOddsPlayerA,MaxOddsPlayerB,AvgOddsPlayerA,AvgOddsPlayerB
0,51,Shanghai,Shanghai Masters,"Sunday, October 12, 2025",Masters 1000,Outdoor,Hard,The Final,3.0,Vacherot V.,...,Rinderknech A.,Vacherot V.,54,204,-150.0,0,1.68,2.38,1.62,2.26
1,51,Shanghai,Shanghai Masters,"Saturday, October 11, 2025",Masters 1000,Outdoor,Hard,Semifinals,3.0,Rinderknech A.,...,Medvedev D.,Rinderknech A.,18,54,-36.0,0,1.40,3.25,1.36,3.09
2,51,Shanghai,Shanghai Masters,"Saturday, October 11, 2025",Masters 1000,Outdoor,Hard,Semifinals,3.0,Vacherot V.,...,Djokovic N.,Vacherot V.,5,204,-199.0,0,1.15,6.50,1.13,5.77
3,51,Shanghai,Shanghai Masters,"Friday, October 10, 2025",Masters 1000,Outdoor,Hard,Quarterfinals,3.0,Rinderknech A.,...,Auger-Aliassime F.,Rinderknech A.,13,54,-41.0,0,1.33,3.75,1.31,3.50
4,51,Shanghai,Shanghai Masters,"Friday, October 10, 2025",Masters 1000,Outdoor,Hard,Quarterfinals,3.0,Medvedev D.,...,De Minaur A.,Medvedev D.,7,18,-11.0,0,1.57,2.65,1.52,2.55


In [ ]:

# WinnerBinary a int
df['WinnerBinary'] = df['WinnerBinary'].astype(int)

# Crear parell ordenat (A < B alfabeticamente)
df['Pair'] = df.apply(
    lambda row: tuple(sorted([row['PlayerA'], row['PlayerB']])), 
    axis=1
)

# Guayador real
df['Ganador'] = df.apply(
    lambda row: row['PlayerA'] if row['WinnerBinary'] == 1 else row['PlayerB'],
    axis=1
)

# Contar H2H
h2h = df.groupby(['Pair', 'Ganador']).size().unstack(fill_value=0)

# Asegurar totes las columnas ---
all_players = pd.Index(df['PlayerA'].unique()).union(df['PlayerB'].unique())
h2h = h2h.reindex(columns=all_players, fill_value=0)

def get_h2h(row):
    pair = row['Pair']
    a = row['PlayerA']
    b = row['PlayerB']
    
    if pair in h2h.index:
        a_wins = h2h.at[pair, a]
        b_wins = h2h.at[pair, b]
    else:
        a_wins = b_wins = 0
    
    return pd.Series([a_wins, b_wins, a_wins - b_wins])

# Aplicar
df[['H2H_A_wins', 'H2H_B_wins', 'H2H_Diff']] = df.apply(get_h2h, axis=1)

# Netejar
df.drop(['Pair', 'Ganador'], axis=1, inplace=True)

# Veure resultats
df[['PlayerA', 'PlayerB', 'RankA', 'RankB', 'WinnerBinary', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'H2H_A_wins', 'H2H_B_wins', 'H2H_Diff']].head()


,PlayerA,PlayerB,RankA,RankB,WinnerBinary,MaxOddsPlayerA,MaxOddsPlayerB,AvgOddsPlayerA,AvgOddsPlayerB,H2H_A_wins,H2H_B_wins,H2H_Diff
0,Rinderknech A.,Vacherot V.,54,204,0,1.68,2.38,1.62,2.26,0,1,-1
1,Medvedev D.,Rinderknech A.,18,54,0,1.40,3.25,1.36,3.09,1,1,0
2,Djokovic N.,Vacherot V.,5,204,0,1.15,6.50,1.13,5.77,0,1,-1
3,Auger-Aliassime F.,Rinderknech A.,13,54,0,1.33,3.75,1.31,3.50,2,1,1
4,De Minaur A.,Medvedev D.,7,18,0,1.57,2.65,1.52,2.55,4,7,-3


In [ ]:
from typing import Any

# Invertim ordre del dataframe, perque volem començar desde el partit més antic
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    # Guardem l'índex original per restaurar l'ordre després
    df['_original_index'] = df.index
    # Ordenar per data ascendent (més antic primer) per processar cronològicament
    df = df.sort_values('Date', ascending=True).reset_index(drop=True)
else:
    print("Warning: No 'Date' column found. Reversing dataframe order.")
    df = df.iloc[::-1].reset_index(drop=True)

# Inicializar diccionaris per comptar victòries i derrotes (es actualitzarà progressivament)
players = pd.concat([df['PlayerA'], df['PlayerB']]).unique()
wins = {player: 0 for player in players}
losses = {player: 0 for player in players}

# Inicializar columnes per comptar victòries i derrotes històrics
df['WinsA'] = 0
df['LossesA'] = 0
df['WinsB'] = 0
df['LossesB'] = 0

# Per cada partit, utilitzar wins/losses històrics (abans d'aquest partit)
# i després actualitzar els contadors per al següent partit
for idx, row in df.iterrows():
    player_a = row['PlayerA']
    player_b = row['PlayerB']
    
    # Asignar wins/losses històrics (abans d'aquest partit)
    df.at[idx, 'WinsA'] = wins[player_a]
    df.at[idx, 'LossesA'] = losses[player_a]
    df.at[idx, 'WinsB'] = wins[player_b]
    df.at[idx, 'LossesB'] = losses[player_b]
    
    # Actualitzar contadors per incloure aquest partit
    winner_binary = row['WinnerBinary']  # 1 si guanya PlayerA, 0 si guanya PlayerB
    
    if winner_binary == 1:
        wins[player_a] += 1
        losses[player_b] += 1
    else:
        wins[player_b] += 1
        losses[player_a] += 1

# Restaurar l'ordre original (més recent primer) si teníem índex original
if '_original_index' in df.columns:
    df = df.sort_values('_original_index', ascending=True).reset_index(drop=True)
    df.drop('_original_index', axis=1, inplace=True)
else:
    # Si no tenia Date, revertir l'ordre que havíem invertit
    df = df.iloc[::-1].reset_index(drop=True)

# Reordenar columnes per a millor visualització
cols_to_move = ['WinsA', 'LossesA', 'WinsB', 'LossesB']
other_cols = [col for col in df.columns if col not in cols_to_move]
df_record_all = df[other_cols[:other_cols.index('PlayerA') + 1] + cols_to_move + other_cols[other_cols.index('PlayerB'):]].copy()

# Calcular win rate (evitar divisió per cero)
df_record_all['WinRateA'] = df_record_all['WinsA'] / (df_record_all['WinsA'] + df_record_all['LossesA']).replace(0, np.nan)
df_record_all['WinRateB'] = df_record_all['WinsB'] / (df_record_all['WinsB'] + df_record_all['LossesB']).replace(0, np.nan)

# Emplenar NaN am 0.5 (win rate neutral) si un jugador no té partits anteriors
df_record_all['WinRateA'] = df_record_all['WinRateA'].fillna(0.5)
df_record_all['WinRateB'] = df_record_all['WinRateB'].fillna(0.5)

df_record_all.head()
#df_record_all[['Date', 'PlayerA', 'PlayerB', 'Surface', 'RankA', 'RankB', 'WinnerBinary', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'H2H_A_wins', 'H2H_B_wins', 'H2H_Diff', 'WinsA', 'LossesA', 'WinsB', 'LossesB', 'WinRateA', 'WinRateB']]

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,WinnerBinary,MaxOddsPlayerA,MaxOddsPlayerB,AvgOddsPlayerA,AvgOddsPlayerB,H2H_A_wins,H2H_B_wins,H2H_Diff,WinRateA,WinRateB
0,51,Shanghai,Shanghai Masters,2025-10-12,Masters 1000,Outdoor,Hard,The Final,3.0,Vacherot V.,...,0,1.68,2.38,1.62,2.26,0,1,-1,0.482412,0.538462
1,51,Shanghai,Shanghai Masters,2025-10-11,Masters 1000,Outdoor,Hard,Semifinals,3.0,Rinderknech A.,...,0,1.40,3.25,1.36,3.09,1,1,0,0.699099,0.479798
2,51,Shanghai,Shanghai Masters,2025-10-11,Masters 1000,Outdoor,Hard,Semifinals,3.0,Vacherot V.,...,0,1.15,6.50,1.13,5.77,0,1,-1,0.835606,0.500000
3,51,Shanghai,Shanghai Masters,2025-10-10,Masters 1000,Outdoor,Hard,Quarterfinals,3.0,Rinderknech A.,...,0,1.33,3.75,1.31,3.50,2,1,1,0.605670,0.477157
4,51,Shanghai,Shanghai Masters,2025-10-10,Masters 1000,Outdoor,Hard,Quarterfinals,3.0,Medvedev D.,...,0,1.57,2.65,1.52,2.55,4,7,-3,0.635071,0.698556


In [43]:
df_record_all[['Date', 'PlayerA', 'PlayerB', 'Surface', 'RankA', 'RankB', 'WinnerBinary', 'MaxOddsPlayerA', 'MaxOddsPlayerB', 'AvgOddsPlayerA', 'AvgOddsPlayerB', 'H2H_A_wins', 'H2H_B_wins', 'H2H_Diff', 'WinsA', 'LossesA', 'WinsB', 'LossesB', 'WinRateA', 'WinRateB']].head(10)

,Date,PlayerA,PlayerB,Surface,RankA,RankB,WinnerBinary,MaxOddsPlayerA,MaxOddsPlayerB,AvgOddsPlayerA,AvgOddsPlayerB,H2H_A_wins,H2H_B_wins,H2H_Diff,WinsA,LossesA,WinsB,LossesB,WinRateA,WinRateB
0,2025-10-12,Rinderknech A.,Vacherot V.,Hard,54,204,0,1.68,2.38,1.62,2.26,0,1,-1,96,103,7,6,0.482412,0.538462
1,2025-10-11,Medvedev D.,Rinderknech A.,Hard,18,54,0,1.40,3.25,1.36,3.09,1,1,0,388,167,95,103,0.699099,0.479798
2,2025-10-11,Djokovic N.,Vacherot V.,Hard,5,204,0,1.15,6.50,1.13,5.77,0,1,-1,1103,217,6,6,0.835606,0.500000
3,2025-10-10,Auger-Aliassime F.,Rinderknech A.,Hard,13,54,0,1.33,3.75,1.31,3.50,2,1,1,235,153,94,103,0.605670,0.477157
4,2025-10-10,De Minaur A.,Medvedev D.,Hard,7,18,0,1.57,2.65,1.52,2.55,4,7,-3,268,154,387,167,0.635071,0.698556
5,2025-10-09,Bergs Z.,Djokovic N.,Hard,44,5,0,3.70,1.35,3.33,1.33,0,1,-1,44,50,1102,217,0.468085,0.835481
6,2025-10-09,Rune H.,Vacherot V.,Hard,11,204,0,1.29,4.00,1.27,3.69,0,1,-1,168,98,5,6,0.631579,0.454545
7,2025-10-08,Medvedev D.,Tien L.,Hard,18,36,1,1.36,3.40,1.35,3.17,1,2,-1,386,167,29,25,0.698011,0.537037
8,2025-10-08,Auger-Aliassime F.,Musetti L.,Hard,13,9,1,2.00,1.91,1.91,1.87,3,3,0,234,153,158,115,0.604651,0.578755
9,2025-10-08,Borges N.,De Minaur A.,Hard,51,7,0,6.00,1.15,5.79,1.13,0,1,-1,64,71,267,154,0.474074,0.634204


---

In [ ]:
from collections import defaultdict

df_grass = df[df['Surface'] == 'Grass'].copy()
print(f"Partits Grass trobats: {len(df_grass)}")

# IMPORTANT: El dataset està ordenat amb el partit més recent primer
# Necessitem processar des de el més antic al més recent per calcular correctament
# Guardar índex original per restaurar l'ordre després
df_grass['_original_index'] = df_grass.index
# Ordenar per data ascendent (més antic primer) per processar cronològicament
df_grass = df_grass.sort_values('Date', ascending=True).reset_index(drop=True)

df_grass['RankA'] = pd.to_numeric(df_grass['RankA'], errors='coerce').fillna(9999)
df_grass['RankB'] = pd.to_numeric(df_grass['RankB'], errors='coerce').fillna(9999)

wins = defaultdict(int)
losses = defaultdict(int)
form10 = defaultdict(lambda: [])
last_match_date = {}

df_grass = df_grass.copy()
df_grass['WinRateA_cum'] = 0.5
df_grass['WinRateB_cum'] = 0.5
df_grass['Form10A'] = 0.5
df_grass['Form10B'] = 0.5
df_grass['RestDaysA'] = 7
df_grass['RestDaysB'] = 7

for idx, row in df_grass.iterrows():
    a, b = row['PlayerA'], row['PlayerB']
    date = row['Date']

    # --- Winrate acumulat en herva finst aquest moment ---
    total_a = wins[a] + losses[a]
    total_b = wins[b] + losses[b]
    df_grass.at[idx, 'WinRateA_cum'] = wins[a] / total_a if total_a > 0 else 0.5
    df_grass.at[idx, 'WinRateB_cum'] = wins[b] / total_b if total_b > 0 else 0.5

    # --- Forma recent (últims 10 partits) ---
    df_grass.at[idx, 'Form10A'] = np.mean(form10[a][-10:]) if form10[a] else 0.5
    df_grass.at[idx, 'Form10B'] = np.mean(form10[b][-10:]) if form10[b] else 0.5

    # --- Díes de descans ---
    last_a = last_match_date.get(a, date - pd.Timedelta(days=30))
    last_b = last_match_date.get(b, date - pd.Timedelta(days=30))
    df_grass.at[idx, 'RestDaysA'] = (date - last_a).days
    df_grass.at[idx, 'RestDaysB'] = (date - last_b).days

    # --- ACTUALIZAR després del partit ---
    if row['WinnerBinary'] == 1:  # guanya A
        wins[a] += 1
        losses[b] += 1
        form10[a].append(1)
        form10[b].append(0)
    else:
        wins[b] += 1
        losses[a] += 1
        form10[b].append(1)
        form10[a].append(0)

    form10[a] = form10[a][-10:]
    form10[b] = form10[b][-10:]

    # Actualitzem ultima data
    last_match_date[a] = date
    last_match_date[b] = date

# Restaurar el orden original (més recent primer)
df_grass = df_grass.sort_values('_original_index', ascending=True).reset_index(drop=True)
df_grass.drop('_original_index', axis=1, inplace=True)

df_grass['DiffWR'] = df_grass['WinRateA_cum'] - df_grass['WinRateB_cum']
df_grass['DiffForm'] = df_grass['Form10A'] - df_grass['Form10B']
df_grass['DiffRest'] = df_grass['RestDaysA'] - df_grass['RestDaysB']
df_grass['LogRankDiff'] = np.log(df_grass['RankA'].clip(1)) - np.log(df_grass['RankB'].clip(1))
df_grass['RankDiff'] = df_grass['RankA'] - df_grass['RankB']
df_grass['H2HDiff'] = df_grass['H2H_A_wins'] - df_grass['H2H_B_wins']

df_grass['ProbA_odds'] = 1 / df_grass['MaxOddsPlayerA'].replace([np.inf, -np.inf], np.nan).fillna(3.0)
df_grass['ProbB_odds'] = 1 / df_grass['MaxOddsPlayerB'].replace([np.inf, -np.inf], np.nan).fillna(3.0)
df_grass['OddsProbDiff'] = df_grass['ProbA_odds'] - df_grass['ProbB_odds']

df_grass['WinRateA_x_Rank'] = df_grass['WinRateA_cum'] / df_grass['RankA'].clip(1, 1000)
df_grass['WinRateB_x_Rank'] = df_grass['WinRateB_cum'] / df_grass['RankB'].clip(1, 1000)
df_grass['EfficiencyDiff']  = df_grass['WinRateA_x_Rank'] - df_grass['WinRateB_x_Rank']


df_grass.head()

Partits Grass trobats: 7734


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,DiffForm,DiffRest,LogRankDiff,H2HDiff,ProbA_odds,ProbB_odds,OddsProbDiff,WinRateA_x_Rank,WinRateB_x_Rank,EfficiencyDiff
0,36,London,Wimbledon,2025-07-13,Grand Slam,Outdoor,Grass,The Final,5.0,Sinner J.,...,0.2,0,0.693147,5,0.483092,0.534759,-0.051668,0.460526,0.736842,-0.276316
1,36,London,Wimbledon,2025-07-11,Grand Slam,Outdoor,Grass,Semifinals,5.0,Alcaraz C.,...,0.1,0,-0.916291,3,0.806452,0.190476,0.615975,0.459459,0.131429,0.328031
2,36,London,Wimbledon,2025-07-11,Grand Slam,Outdoor,Grass,Semifinals,5.0,Sinner J.,...,0.1,0,1.791759,-1,0.327869,0.680272,-0.352403,0.145238,0.729730,-0.584492
3,36,London,Wimbledon,2025-07-09,Grand Slam,Outdoor,Grass,Quarterfinals,5.0,Sinner J.,...,-0.2,0,2.302585,-5,0.256410,0.746269,-0.489858,0.054545,0.722222,-0.667677
4,36,London,Wimbledon,2025-07-09,Grand Slam,Outdoor,Grass,Quarterfinals,5.0,Djokovic N.,...,-0.2,0,1.386294,-2,0.116279,0.892857,-0.776578,0.025000,0.145084,-0.120084


In [45]:
features = [
    'WinnerBinary',
    'LogRankDiff',      # ranking
    'DiffWR',           # diferencia de winrate históric en herva
    'DiffForm',         # últims 10
    'H2HDiff',          # head-to-head
    'OddsProbDiff',     # diferencia de probabilitat implícita de quotas
    'DiffRest',         # dies de rest
    'EfficiencyDiff',   # winrate / rank → jugador que rendeix per sobre del seu ranking
    'RankDiff'          # diferencia de ranking
]

df_grass[features].head(10)

,WinnerBinary,LogRankDiff,DiffWR,DiffForm,H2HDiff,OddsProbDiff,DiffRest,EfficiencyDiff,RankDiff
0,0,0.693147,0.184211,0.2,5,-0.051668,0,-0.276316,1.0
1,1,-0.916291,0.261776,0.1,3,0.615975,0,0.328031,-3.0
2,0,1.791759,0.141699,0.1,-1,-0.352403,0,-0.584492,5.0
3,0,2.302585,-0.176768,-0.2,-5,-0.489858,0,-0.667677,9.0
4,0,1.386294,-0.270504,-0.2,-2,-0.776578,0,-0.120084,18.0
5,1,-1.386294,-0.000887,0.1,0,0.525264,0,0.097782,-15.0
6,1,-3.417727,0.363475,0.4,3,0.860505,0,0.449265,-59.0
7,0,0.606136,-0.215020,-0.2,-4,-0.663842,0,-0.085423,5.0
8,0,3.044522,-0.109347,-0.1,-4,-0.897010,0,-0.685479,20.0
9,1,-1.547563,-0.041408,0.0,3,0.525018,0,0.040355,-37.0


In [48]:
df_grass.to_csv('wimbeldon_dataset.csv', index=False)